## Iteration 2: includes four type of stroke patients and four arrival functions

## **. Patient Admission Sources**  
Patients enter the hospital through **two primary pathways**:  
- **New Admissions**: Direct hospital entry for Stroke, TIA, Complex Neurological, and Other Medical Cases.  


| Patient Type                     | Admissions (n) | Percentage (%) |
|----------------------------------|---------------|--------------|
| Stroke                           | 1,320         | 54%          |
| Transient Ischemic Attack (TIA)  | 158           | 6%           |
| Complex Neurological             | 456           | 19%          |
| Other Medical Cases              | 510           | 21%          |

**Patient admissions are distributed as follows**

| Category                | Mean(Days) |
|-------------------------|----------------------------|
| Stroke                  | 1.2                        |
| TIA (Transient Ischemic Attack) | 9.3                |
| Complex Neurological    | 3.6                        |
| Other                   | 3.2                        |



## 1. Imports 

In [1]:
import numpy as np
import itertools
import simpy
import sys
import os

In [2]:
# Add parent directory to path so we can import distribution.py
sys.path.append(os.path.abspath(".."))  # noqa: E402

from distribution import Exponential  # noqa: E402

## 2. Constants

In [3]:
# default mean inter-arrival times(exp)
IAT_STROKE = 1.2
IAT_TIA = 9.3
IAT_COMPLEX_NEURO = 3.6
IAT_OTHER = 3.2

# sampling settings
N_STREAMS = 4
DEFAULT_RND_SET = 0

# Boolean switch to simulation results as the model runs
TRACE = False

# run variables (units = days)
RUN_LENGTH = 365 * 3

## 2. Helper classes and functions

In [4]:
def trace(msg):
    """
    Turing printing of events on and off.

    Params:
    -------
    msg: str
        string to print to screen.
    """
    if TRACE:
        print(msg)

## 3. Experiment class

In [5]:
class Experiment:
    """
    Encapsulates the concept of an experiment 🧪 for the Orthopedic Surgery
    trauma arrival simulator. Manages parameters, PRNG streams and results.
    """

    def __init__(
        self,
        random_number_set=DEFAULT_RND_SET,
        n_streams=N_STREAMS,
        iat_stroke=IAT_STROKE,
        iat_tia=IAT_TIA,
        iat_complex_neuro=IAT_COMPLEX_NEURO,
        iat_other=IAT_OTHER,
    ):
        """
        The init method sets up our defaults.
        """
        # sampling
        self.random_number_set = random_number_set
        self.n_streams = n_streams

        # store parameters for the run of the model
        self.iat_stroke = iat_stroke
        self.iat_tia = iat_tia
        self.iat_complex_neuro = iat_complex_neuro
        self.iat_other = iat_other

        # initialise results to zero
        self.init_results_variables()

        # initialise sampling objects
        self.init_sampling()

    def set_random_no_set(self, random_number_set):
        """
        Controls the random sampling
        Parameters:
        ----------
        random_number_set: int
            Used to control the set of pseudo random numbers used by
            the distributions in the simulation.
        """
        self.random_number_set = random_number_set
        self.init_sampling()

    def init_sampling(self):
        """
        Create the distributions used by the model and initialise
        the random seeds of each.
        """
        # produce n non-overlapping streams
        seed_sequence = np.random.SeedSequence(self.random_number_set)
        self.seeds = seed_sequence.spawn(self.n_streams)

        # create distributions

        # inter-arrival time distributions
        self.arrival_stroke = Exponential(
            self.iat_stroke, random_seed=self.seeds[0]
        )

        self.arrival_tia = Exponential(self.iat_tia, random_seed=self.seeds[0])

        self.arrival_complex_neuro = Exponential(
            self.iat_complex_neuro, random_seed=self.seeds[0]
        )

        self.arrival_other = Exponential(
            self.iat_other, random_seed=self.seeds[0]
        )

    def init_results_variables(self):
        """
        Initialise all of the experiment variables used in results
        collection.  This method is called at the start of each run
        of the model
        """
        # variable used to store results of experiment
        self.results = {}
        self.results["n_stroke"] = 0
        self.results["n_tia"] = 0
        self.results["n_complex_neuro"] = 0
        self.results["n_other"] = 0
        self.results["n_total"] = 0

## 4. A function per arrival source

The first approach we will use is creating an arrival generator per source.  There will be some code redundancy, but it will a clear design for others to understand.

In [6]:
def stroke_arrivals_generator(env, args):
    """
    Arrival process for strokes.

    Parameters:
    ------
    env: simpy.Environment
        The simpy environment for the simulation

    args: Experiment
        The settings and input parameters for the simulation.
    """
    # use itertools as it provides an infinite loop
    # with a counter variable that we can use for unique Ids
    for patient_count in itertools.count(start=1):

        # the sample distribution is defined by the experiment.
        inter_arrival_time = args.arrival_stroke.sample()
        yield env.timeout(inter_arrival_time)

        args.results["n_stroke"] = patient_count
        args.results["n_total"] += 1

        trace(f"{env.now:.2f}: STROKE arrival.")

In [7]:
def tia_arrivals_generator(env, args):
    """
    Arrival process for TIAs.

    Parameters:
    ------
    env: simpy.Environment
        The simpy environment for the simulation

    args: Experiment
        The settings and input parameters for the simulation.
    """
    # use itertools as it provides an infinite loop
    # with a counter variable that we can use for unique Ids
    for patient_count in itertools.count(start=1):

        # the sample distribution is defined by the experiment.
        inter_arrival_time = args.arrival_tia.sample()
        yield env.timeout(inter_arrival_time)

        args.results["n_tia"] = patient_count
        args.results["n_total"] += 1
        trace(f"{env.now:.2f}: TIA arrival.")

In [8]:
def complex_neuro_arrivals_generator(env, args):
    """
    Arrival process for complex neuro cases.

    Parameters:
    ------
    env: simpy.Environment
        The simpy environment for the simulation

    args: Experiment
        The settings and input parameters for the simulation.
    """
    # use itertools as it provides an infinite loop
    # with a counter variable that we can use for unique Ids
    for patient_count in itertools.count(start=1):

        # the sample distribution is defined by the experiment.
        inter_arrival_time = args.arrival_complex_neuro.sample()
        yield env.timeout(inter_arrival_time)

        args.results["n_complex_neuro"] = patient_count
        args.results["n_total"] += 1
        trace(f"{env.now:.2f}: COMPLEX NEURO arrival.")

In [9]:
def other_arrivals_generator(env, args):
    """
    Arrival process for other cases.

    Parameters:
    ------
    env: simpy.Environment
        The simpy environment for the simulation

    args: Experiment
        The settings and input parameters for the simulation.
    """
    # use itertools as it provides an infinite loop
    # with a counter variable that we can use for unique Ids
    for patient_count in itertools.count(start=1):

        # the sample distribution is defined by the experiment.
        inter_arrival_time = args.arrival_other.sample()
        yield env.timeout(inter_arrival_time)

        args.results["n_other"] = patient_count
        args.results["n_total"] += 1
        trace(f"{env.now:.2f}: OTHER arrival.")

## 5. Single run function

In [10]:
def single_run(experiment, rep=0, run_length=RUN_LENGTH):
    """
    Perform a single run of the model and return the results

    Parameters:
    -----------

    experiment: Experiment
        The experiment/parameters to use with model

    rep: int
        The replication number.

    run_length: float, optional (default=RUN_LENGTH)
        The run length of the model
    """

    # reset all results variables to zero and empty
    experiment.init_results_variables()

    # set random number set to the replication no.
    # this controls sampling for the run.
    experiment.set_random_no_set(rep)

    # environment is (re)created inside single run
    env = simpy.Environment()

    # we pass all arrival generators to simpy
    env.process(stroke_arrivals_generator(env, experiment))
    env.process(tia_arrivals_generator(env, experiment))
    env.process(complex_neuro_arrivals_generator(env, experiment))
    env.process(other_arrivals_generator(env, experiment))

    # run for warm-up + results collection period
    env.run(until=run_length)

    # return the count of the arrivals
    return experiment.results

In [11]:
TRACE = False
experiment = Experiment()
results = single_run(experiment)

In [12]:
# the results of patient arrival
results

{'n_stroke': 934,
 'n_tia': 113,
 'n_complex_neuro': 311,
 'n_other': 358,
 'n_total': 1716}

### Test percentage of the patients with stroke who are admitted to the stroke unit.

In [13]:
# percentage of patients with stroke who are admitted to the stroke unit
stroke_percentage = results["n_stroke"] / results["n_total"]
tia_percentage = results["n_tia"] / results["n_total"]
complex_percentage = results["n_complex_neuro"] / results["n_total"]
other_percentage = results["n_other"] / results["n_total"]

print(f"% stroke: {stroke_percentage*100:.2f}%")
print(f"% tia : {tia_percentage*100:.2f}%")
print(f"% complex neuro: {complex_percentage*100:.2f}%")
print(f"% Others: {other_percentage*100:.2f}%")

% stroke: 54.43%
% tia : 6.59%
% complex neuro: 18.12%
% Others: 20.86%


## A hospital that only provides surgery for hip fractures

In [14]:
M = 1_000_000
experiment = Experiment(iat_stroke=M, iat_complex_neuro=M, iat_other=M)
results = single_run(experiment)
results

{'n_stroke': 0,
 'n_tia': 113,
 'n_complex_neuro': 0,
 'n_other': 0,
 'n_total': 113}